In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor  # Import du modèle Random Forest
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_squared_error, r2_score

# Chargement des données
df = pd.read_csv("train_data.csv")  

# Calcul du prix moyen par zipcode
df["prix_moy_zipcode"] = df.groupby("zipcode")["prix"].transform("mean")

# Sélection des variables explicatives
X = df.drop(columns=["prix", "id", "date", "zipcode", "lat", "long"])  
y = df["prix"]

# Train/test split (20% pour le test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline avec Random Forest
pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),  # Imputation des valeurs manquantes
    ("scaler", StandardScaler()),  # Standardisation (pas obligatoire pour RF, mais utile pour SelectKBest)
    ("feature_selection", SelectKBest(score_func=f_regression, k=5)),  # Sélection des meilleures variables
    ("model", RandomForestRegressor(random_state=42, n_jobs=-1))  # Modèle Random Forest
])

# Entraînement du modèle
pipeline.fit(X_train, y_train)

# Prédictions
y_pred = pipeline.predict(X_test)

# Évaluation du modèle
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"RMSE du modèle Random Forest : {rmse:.2f}")
print(f"R² du modèle Random Forest : {r2:.3f}")

# GridSearchCV pour optimiser les hyperparamètres de Random Forest
param_grid = {
    "feature_selection__k": [3, 5, 7, 9, 11],  # Nombre de variables sélectionnées
    "model__n_estimators": [50, 100, 200],  # Nombre d'arbres dans la forêt
    "model__max_depth": [None, 10, 20, 30]  # Profondeur maximale de l'arbre
}

# Recherche des meilleurs hyperparamètres avec validation croisée
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring="neg_root_mean_squared_error", n_jobs=-1)
grid_search.fit(X_train, y_train)

# Meilleurs paramètres et scores optimisés
best_rmse = -grid_search.best_score_  # Inversion car GridSearchCV retourne une valeur négative
best_r2 = grid_search.best_estimator_.score(X_test, y_test)  # Score R² du meilleur modèle



print(f"Meilleurs paramètres : {grid_search.best_params_}")
print(f"Meilleur RMSE en validation croisée : {best_rmse:.2f}")
print(f"R² du meilleur modèle : {best_r2:.3f}")


RMSE du modèle Random Forest : 166804.49
R² du modèle Random Forest : 0.798
Meilleurs paramètres : {'feature_selection__k': 11, 'model__max_depth': 30, 'model__n_estimators': 100}
Meilleur RMSE en validation croisée : 140520.75
R² du meilleur modèle : 0.844


In [4]:
best_model=grid_search.best_estimator_

# Charger les données de test
df_test = pd.read_csv("test_data.csv")

# Charger les données d'entraînement pour récupérer les moyennes par zipcode
df_train = pd.read_csv("train_data.csv")
prix_moy_zipcode = df_train.groupby("zipcode")["prix"].mean()

# Ajouter la colonne prix moyen par zipcode dans df_test (basé sur df_train)
df_test["prix_moy_zipcode"] = df_test["zipcode"].map(prix_moy_zipcode)

# Remplacer les NaN (au cas où certains zipcodes de test ne sont pas dans train)
df_test["prix_moy_zipcode"].fillna(df_train["prix"].mean(), inplace=True)

# Supprimer les colonnes non utilisées dans la prédiction
X_test_final = df_test.drop(columns=["id", "date", "zipcode", "lat", "long"])

# Prédiction avec le meilleur modèle entraîné
y_pred_test = best_model.predict(X_test_final)

# Créer un DataFrame pour soumission Kaggle
submission = pd.DataFrame({"id": df_test["id"], "prix": y_pred_test})

# Sauvegarder en CSV
submission.to_csv("submission.csv", index=False)

print("📂 Fichier 'submission.csv' généré avec succès !")

📂 Fichier 'submission.csv' généré avec succès !


C:\Users\qevan\AppData\Local\Temp\ipykernel_20340\134256900.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_test["prix_moy_zipcode"].fillna(df_train["prix"].mean(), inplace=True)
